# GeekBrains
## Машинное обучение в бизнесе
## ДЗ Урока 4 (Uplift-моделирование)
### Виталий Казанцев

#### Т/з
1. скачать набор данных маркетинговых кампаний отсюда https://www.kaggle.com/davinwijaya/customer-retention
2. там поле conversion - это целевая переменная, а offer - коммуникация. Переименовать поля (conversion -> target, offer -> treatment) и привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.
3. сделать разбиение набора данных не тренировочную и тестовую выборки
4. сделать feature engineering на ваше усмотрение (допускается свобода выбора методов)
5. провести uplift-моделирование 3 способами: одна модель с признаком коммуникации (S learner), модель с трансформацией таргета (трансформация классов п. 2. 1) и вариант с двумя независимыми моделями
6. в конце вывести единую таблицу сравнения метрик uplift@10%, uplift@20% этих 3 моделей
7. построить модель UpliftTreeClassifier и попытаться описать словами полученное дерево
8. (опционально) для модели S learner (модель с дополнительным признаком коммуникации) построить зависимость таргета (конверсии - поле conversion) от значения uplift: 1) сделать прогноз и получить uplift для тестовой выборки 2) отсортировать тестовую выборку по uplift по убыванию 3) разбить на децили (pandas qcut вам в помощь) 4) для каждого дециля посчитать среднюю conversion
9. (опционально) построить модель UpliftRandomForestClassifier и попытаться описать словами полученное дерево

__Задание 1__  
скачать набор данных маркетинговых кампаний отсюда https://www.kaggle.com/davinwijaya/customer-retention

In [1]:
import pandas as pd
#pd.set_option('display.max_columns', None)
import numpy as np

from sklearn.model_selection import train_test_split

from sklift.metrics import uplift_at_k
from sklift.viz import plot_uplift_preds
from sklift.models import SoloModel, ClassTransformation, TwoModels

from catboost import CatBoostClassifier

%matplotlib inline

In [2]:
df = pd.read_csv('data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64000 entries, 0 to 63999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   recency        64000 non-null  int64  
 1   history        64000 non-null  float64
 2   used_discount  64000 non-null  int64  
 3   used_bogo      64000 non-null  int64  
 4   zip_code       64000 non-null  object 
 5   is_referral    64000 non-null  int64  
 6   channel        64000 non-null  object 
 7   offer          64000 non-null  object 
 8   conversion     64000 non-null  int64  
dtypes: float64(1), int64(5), object(3)
memory usage: 4.4+ MB


In [3]:
df.sample(5)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
58691,3,161.22,1,0,Urban,0,Web,No Offer,0
19086,1,699.31,0,1,Surburban,1,Multichannel,Discount,0
18044,6,301.67,0,1,Surburban,0,Phone,No Offer,0
51102,2,163.74,1,0,Urban,0,Web,No Offer,0
44733,5,480.84,1,0,Urban,0,Web,Discount,0


In [4]:
df.shape

(64000, 9)

__Задание 2__  
там поле conversion - это целевая переменная, а offer - коммуникация. Переименовать поля (conversion -> target, offer -> treatment) и привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.

In [5]:
df.rename(columns={'conversion': 'target', 'offer': 'treatment'}, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64000 entries, 0 to 63999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   recency        64000 non-null  int64  
 1   history        64000 non-null  float64
 2   used_discount  64000 non-null  int64  
 3   used_bogo      64000 non-null  int64  
 4   zip_code       64000 non-null  object 
 5   is_referral    64000 non-null  int64  
 6   channel        64000 non-null  object 
 7   treatment      64000 non-null  object 
 8   target         64000 non-null  int64  
dtypes: float64(1), int64(5), object(3)
memory usage: 4.4+ MB


In [6]:
df.loc[(df.treatment != 'No Offer'), 'treatment'] = 1
df.loc[(df.treatment == 'No Offer'), 'treatment'] = 0
df = df.astype({'treatment': np.int8})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64000 entries, 0 to 63999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   recency        64000 non-null  int64  
 1   history        64000 non-null  float64
 2   used_discount  64000 non-null  int64  
 3   used_bogo      64000 non-null  int64  
 4   zip_code       64000 non-null  object 
 5   is_referral    64000 non-null  int64  
 6   channel        64000 non-null  object 
 7   treatment      64000 non-null  int8   
 8   target         64000 non-null  int64  
dtypes: float64(1), int64(5), int8(1), object(2)
memory usage: 4.0+ MB


In [7]:
df.sample(5)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
21590,10,67.39,0,1,Surburban,1,Phone,0,0
60689,11,178.64,0,1,Surburban,1,Phone,1,1
27668,5,463.36,1,1,Urban,0,Web,1,0
21026,11,70.05,1,0,Surburban,1,Web,1,0
30957,6,29.99,0,1,Surburban,1,Web,0,0


__Задание 3__  
сделать разбиение набора данных не тренировочную и тестовую выборки

In [8]:
X_train, X_test, y_train, y_test, treat_train, treat_test = train_test_split(df.drop(columns=['target']), 
                                                            df['target'],
                                                            df['treatment'],
                                                            random_state=0)

X_train.shape, X_test.shape, y_train.shape, y_test.shape, treat_train.shape

((48000, 8), (16000, 8), (48000,), (16000,), (48000,))

__Задание 4__  
сделать feature engineering на ваше усмотрение (допускается свобода выбора методов)

Разделим пользователей по объему расходов(expenses)

In [9]:
def expenses(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['is_whale'] = 0
    df['is_dolphin'] = 0
    df['is_minnow'] = 0
    df.loc[df['history'] >= 1000, 'is_whale'] = 1
    df.loc[(df['history'] < 1000) & (df['history'] >= 200), 'is_dolphin'] = 1
    df.loc[df['history'] < 200, 'is_minnow'] = 1
    return df

In [10]:
X_train = expenses(X_train)
X_test = expenses(X_test)

In [11]:
X_train.sample(5)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,is_whale,is_dolphin,is_minnow
22310,5,724.57,0,1,Surburban,1,Multichannel,0,0,1,0
13693,2,175.94,1,0,Surburban,1,Phone,0,0,0,1
31533,3,149.31,0,1,Surburban,0,Web,1,0,0,1
31785,2,206.93,0,1,Surburban,0,Web,0,0,1,0
40634,1,56.16,1,0,Urban,0,Phone,0,0,0,1


__Задание 5__  
провести uplift-моделирование 3 способами: одна модель с признаком коммуникации (S learner), модель с трансформацией таргета (трансформация классов п. 2. 1) и вариант с двумя независимыми моделями  
__Задание 6__  
в конце вывести единую таблицу сравнения метрик uplift@10%, uplift@20% этих 3 моделей


In [12]:
metrics_df = pd.DataFrame(columns=['uplift@10%', 'uplift@20%'])

In [13]:
cat_features = ['zip_code', 'channel']

In [14]:
sm = SoloModel(CatBoostClassifier(iterations=20, thread_count=2, random_state=42, silent=True))
sm = sm.fit(X_train, y_train, treat_train, estimator_fit_params={'cat_features': cat_features})

uplift_sm = sm.predict(X_test)
print(uplift_sm)

sm_score_10 = uplift_at_k(y_true=y_test, uplift=uplift_sm, treatment=treat_test, strategy='by_group', k=0.1)
sm_score_20 = uplift_at_k(y_true=y_test, uplift=uplift_sm, treatment=treat_test, strategy='by_group', k=0.2)

metrics_df = metrics_df.append({'uplift@10%': sm_score_10, 'uplift@20%': sm_score_20}, ignore_index=True)

[0.06326457 0.06920032 0.07275016 ... 0.06685124 0.02610939 0.07757569]


In [15]:
ct = ClassTransformation(CatBoostClassifier(iterations=20, thread_count=2, random_state=42, silent=True))
ct = ct.fit(X_train, y_train, treat_train, estimator_fit_params={'cat_features': cat_features})

uplift_ct = ct.predict(X_test)
print(uplift_ct)

ct_score_10 = uplift_at_k(y_true=y_test, uplift=uplift_ct, treatment=treat_test, strategy='by_group', k=0.1)
ct_score_20 = uplift_at_k(y_true=y_test, uplift=uplift_ct, treatment=treat_test, strategy='by_group', k=0.2)

metrics_df = metrics_df.append({'uplift@10%': ct_score_10, 'uplift@20%': ct_score_20}, ignore_index=True)

[-0.68133514 -0.66582944 -0.61137839 ...  0.78141424 -0.65206333
 -0.32893603]


In [16]:
tm = TwoModels(
    estimator_trmnt=CatBoostClassifier(iterations=20, thread_count=2, random_state=42, silent=True), 
    estimator_ctrl=CatBoostClassifier(iterations=20, thread_count=2, random_state=42, silent=True), 
    method='vanilla'
)
tm = tm.fit(
    X_train, y_train, treat_train,
    estimator_trmnt_fit_params={'cat_features': cat_features}, 
    estimator_ctrl_fit_params={'cat_features': cat_features}
)

uplift_tm = tm.predict(X_test)
print(uplift_tm)

tm_score_10 = uplift_at_k(y_true=y_test, uplift=uplift_tm, treatment=treat_test, strategy='by_group', k=0.1)
tm_score_20 = uplift_at_k(y_true=y_test, uplift=uplift_tm, treatment=treat_test, strategy='by_group', k=0.2)

metrics_df = metrics_df.append({'uplift@10%': tm_score_10, 'uplift@20%': tm_score_20}, ignore_index=True)

[0.08574949 0.08029174 0.09133908 ... 0.07039956 0.0432995  0.12032493]


In [18]:
metrics_df.index = ['Solo Learner', 'Transform Learner', 'Two Model Learner']
metrics_df

,uplift@10%,uplift@20%
Solo Learner,0.059370,0.068407
Transform Learner,0.225533,0.195837
Two Model Learner,0.079349,0.065813


__Задание 7__  
построить модель UpliftTreeClassifier и попытаться описать словами полученное дерево

In [19]:
%%time
from IPython.display import Image
from causalml.inference.tree import UpliftTreeClassifier, UpliftRandomForestClassifier
from causalml.inference.tree import uplift_tree_string, uplift_tree_plot

Failed to import duecredit due to No module named 'duecredit'
/home/vitaly/anaconda3/envs/venvLDS/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: sklearn.tree._tree.TreeBuilder size changed, may indicate binary incompatibility. Expected 80 from C header, got 72 from PyObject